In [1]:
%load_ext autoreload
%autoreload 2

# Detecting audio issues in a condition monitoring dataset (audio)
This notebook aims at detecting issues in a **condition monitoring** dataset using **audio data**. As a basis it uses the DCASE challenge dataset where the goal is to detect if a machine is in a defect state or not.

In order to run the example install the **dependencies** as follows:

In [1]:
!pip install datasets


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


## Step 1: Download the dataset

In [2]:
# Import datasets to download the data
import datasets

In [3]:
# Download the dataset and convert to pandas dataframe
dataset = datasets.load_dataset(
        "renumics/dcase23-task2-enriched", "dev", split="all", streaming=False
    )
df = dataset.to_pandas()

Found cached dataset dcase23-task2-enriched (/home/daniel/.cache/huggingface/datasets/renumics___dcase23-task2-enriched/dev/3.0.0/c945b4ebc3162e1b75113e4813bc56f4eceed94d7e693d03c39f898c32d751bb)


In [4]:
# Sample the dataset randomly to make the example run faster
df = df.sample(1000)

# Step 2: Detect problematic data slices based on audio data

In [5]:
# The imports
from sklearn.metrics import accuracy_score
from renumics.spotlight import Audio
from sliceguard import SliceGuard

In [24]:
# Run slice detection based on general purpose audio embeddings (pretrained model trained on Audioset)
sg = SliceGuard()
issue_df = sg.find_issues(
    df,
    ["path"],
    "label",
    "dev_train_lof_anomaly",
    accuracy_score,
    metric_mode="max",
    min_support=5,
    min_drop=0.2
)

Feature path was inferred as referring to raw data. If this is not the case, please specify in feature_types!
Using default model for computing embeddings for feature path.


Some weights of the model checkpoint at MIT/ast-finetuned-audioset-10-10-0.4593 were not used when initializing ASTModel: ['classifier.dense.weight', 'classifier.layernorm.weight', 'classifier.dense.bias', 'classifier.layernorm.bias']
- This IS expected if you are initializing ASTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ASTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

The overall metric value is 0.868
Using 5 as minimum support for determining problematic clusters.
Using 0.2 as minimum drop for determining problematic clusters.
Identified 7 problematic slices.


In [25]:
# Report the results in Renumics Spotlight
sg.report(spotlight_dtype={"path": Audio})

,audio,path,section,domain,label,class,d1p,d1v,d2p,d2v,...,d3v,dev_train_lof_anomaly,dev_train_lof_anomaly_score,add_train_lof_anomaly,add_train_lof_anomaly_score,embeddings_ast-finetuned-audioset-10-10-0.4593,issue,issue_metric,issue_explanation,sg_emb_path
2387,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,5,car,C1,spd,7,...,1.0,0,1.029181,1,1.556761,"[2.6978528, 0.6663538, -2.8895288, 1.8184402, ...",-1,NaN,,"[2.581609010696411, 0.8865570425987244, -2.506..."
6222,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,4,car,B1,spd,40V,...,1.0,0,1.103587,1,2.177201,"[1.5311544, -1.239571, 2.4675872, 0.8665277, -...",-1,NaN,,"[1.7293566465377808, -0.5932053327560425, 2.94..."
3782,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,2,vel,16,loc,A,...,nan,0,1.188019,1,1.343164,"[1.296662, 1.5886989, -1.8994781, 2.01296, -1....",-1,NaN,,"[1.187314510345459, 1.7998664379119873, -1.535..."
478,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,6,pat,1,nan,nan,...,nan,0,1.096836,1,3.408961,"[0.32793164, 0.39336237, -1.7705992, 0.2071408...",-1,NaN,,"[0.1500183343887329, 0.6747907996177673, -1.99..."
3126,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,2,vel,21,loc,A,...,nan,0,0.994130,1,2.017463,"[-0.5759594, 2.0182471, -0.88640237, 0.8374698...",-1,NaN,,"[-0.7415647506713867, 2.4746668338775635, -0.7..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4963,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,1,m-n,X,nan,nan,...,nan,0,1.066225,1,1.313458,"[0.8141135, 0.1499078, -0.42280692, 0.23543704...",-1,NaN,,"[0.8595283031463623, 0.1479468196630478, -0.18..."
7026,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,1,6,pat,0,nan,nan,...,nan,0,1.297377,1,1.700536,"[1.6392725, 1.2710798, -0.3470587, 1.0596004, ...",-1,NaN,,"[1.7499339580535889, 1.6953036785125732, -0.24..."
2946,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,5,car,D1,spd,10,...,1.0,0,1.004022,1,2.221871,"[1.5651789, 0.2712275, -0.81729484, 0.76568, -...",-1,NaN,,"[1.4127410650253296, 0.3044147789478302, -0.28..."
1053,"{'bytes': None, 'path': '/home/daniel/.cache/h...",/home/daniel/.cache/huggingface/datasets/downl...,0,0,0,0,volt,1.5,wt,150,...,nan,0,1.091428,1,1.454058,"[1.604075, 1.1287867, -0.0003990084, 0.7226964...",-1,NaN,,"[1.9818440675735474, 1.4262199401855469, 0.265..."
